In [2]:
#Importamos las librerias necesarias para el etl.
import pandas as pd
import os
from pathlib import Path

In [3]:
#Cargamos el archivo csv en un dataframe de pandas.

import pandas as pd

# Obtener la ruta del archivo .py
current_dir = Path(__file__).resolve().parent

# Construir la ruta relativa al archivo CSV
csv_path = current_dir.parents[1] / 'data' / 'raw' / 'scraped_earthquakes.csv'

# Leer el archivo CSV
df = pd.read_csv(csv_path)

#Mostramos las primeras filas del dataframe.
df.head()

NameError: name '__file__' is not defined

In [22]:
import os
from pathlib import Path
import pandas as pd
import numpy as np

# Obtener la ruta del directorio actual (sin usar __file__)
current_dir = Path(os.getcwd())  # Obtiene el directorio actual de trabajo

# Construir la ruta relativa al archivo CSV
csv_path = current_dir.parents[1] / 'data' / 'raw' / 'scraped_earthquakes.csv'

# Cargar el CSV
df = pd.read_csv(csv_path)





In [23]:
#mostrar la longitud de el dataframe.
print(len(df))

1350347


In [24]:
#Mostramos los tipos de datos de cada columna.
df.dtypes


Date & Time UTC     object
Lat. degrees       float64
Lon. degrees       float64
Depth km             int64
Region              object
Type                object
A/M                 object
Magnitude          float64
Network             object
dtype: object

In [25]:
#convertimos la columna Date & Time UTC a un objeto datetime.
df['Date & Time UTC'] = pd.to_datetime(df['Date & Time UTC'], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')

df.head(5)

,Date & Time UTC,Lat. degrees,Lon. degrees,Depth km,Region,Type,A/M,Magnitude,Network
0,2025-04-01 23:59:55.800,26.467,128.814,10,"RYUKYU ISLANDS, JAPAN",origin,A,5.1,SC4
1,2025-04-01 23:59:55.000,26.210,128.630,10,"RYUKYU ISLANDS, JAPAN",origin,A,5.1,BEO
2,2025-04-01 23:59:54.700,26.118,128.556,10,"RYUKYU ISLANDS, JAPAN",origin,M,5.1,GFZ
3,2025-04-01 23:59:54.100,26.060,128.500,10,"RYUKYU ISLANDS, JAPAN",origin,M,5.1,AUST
4,2025-04-01 23:59:53.700,26.020,128.510,10,"RYUKYU ISLANDS, JAPAN",origin,M,5.2,GSRAS


In [26]:
# Intentamos dividir la columna 'Region' por la coma y el espacio
split_columns = df['Region'].str.split(', ', expand=True)


# Verificamos si la división fue exitosa
if 1 in split_columns.columns:
    df['Country'] = split_columns[1]
else:
    df['Country'] = split_columns[0]  # Si no hubo división, mantenemos el valor original en 'Country'

df['Region'] = split_columns[0].fillna(df['Region'])  # Si no hubo división, mantenemos el valor original en 'Region'



In [27]:
#Eliminamos columnas innecesarias.
clean_df = df.drop(columns=['Type', 'Network', 'A/M'])
clean_df.head(5)

,Date & Time UTC,Lat. degrees,Lon. degrees,Depth km,Region,Magnitude,Country
0,2025-04-01 23:59:55.800,26.467,128.814,10,RYUKYU ISLANDS,5.1,JAPAN
1,2025-04-01 23:59:55.000,26.210,128.630,10,RYUKYU ISLANDS,5.1,JAPAN
2,2025-04-01 23:59:54.700,26.118,128.556,10,RYUKYU ISLANDS,5.1,JAPAN
3,2025-04-01 23:59:54.100,26.060,128.500,10,RYUKYU ISLANDS,5.1,JAPAN
4,2025-04-01 23:59:53.700,26.020,128.510,10,RYUKYU ISLANDS,5.2,JAPAN


In [28]:
#Eliminamos todos los registros que no tienen coordenadas o magnitud.
clean_df = clean_df.dropna(subset=['Lat. degrees', 'Lon. degrees', 'Magnitude'])

In [32]:
n = len(df)

# Generamos valores aleatorios pequeños para ajustar (entre 0 y 1e-5)
ajuste_lat = np.random.uniform(0, 1e-5, n)
ajuste_lon = np.random.uniform(0, 1e-5, n)

# Convertimos las columnas originales a float (por si acaso)
clean_df['Lat. degrees'] = df['Lat. degrees'].astype(float)
clean_df['Lon. degrees'] = df['Lon. degrees'].astype(float)

# Creamos las nuevas columnas ajustadas
clean_df['Lat_adj'] = df['Lat. degrees'] + ajuste_lat
clean_df['Lon_adj'] = df['Lon. degrees'] + ajuste_lon

df

,Date & Time UTC,Lat. degrees,Lon. degrees,Depth km,Region,Type,A/M,Magnitude,Network,Country,Lat_adj,Lon_adj
0,2025-04-01 23:59:55.800,26.467,128.814,10,RYUKYU ISLANDS,origin,A,5.1,SC4,JAPAN,26.467000,128.814001
1,2025-04-01 23:59:55.000,26.210,128.630,10,RYUKYU ISLANDS,origin,A,5.1,BEO,JAPAN,26.210008,128.630003
2,2025-04-01 23:59:54.700,26.118,128.556,10,RYUKYU ISLANDS,origin,M,5.1,GFZ,JAPAN,26.118004,128.556008
3,2025-04-01 23:59:54.100,26.060,128.500,10,RYUKYU ISLANDS,origin,M,5.1,AUST,JAPAN,26.060007,128.500007
4,2025-04-01 23:59:53.700,26.020,128.510,10,RYUKYU ISLANDS,origin,M,5.2,GSRAS,JAPAN,26.020002,128.510006
...,...,...,...,...,...,...,...,...,...,...,...,...
1350342,2006-01-30 20:12:50.200,46.090,27.631,10,ROMANIA,origin,A,3.7,NIEP,None,46.090003,27.631003
1350343,2006-01-30 20:12:48.200,45.720,28.560,0,UKRAINE-ROMANIA-MOLDOVA BDR REG,origin,M,3.6,GSRAS,None,45.720009,28.560009
1350344,2006-01-30 20:12:46.500,45.948,27.956,10,ROMANIA,origin,M,4.1,NIEP,None,45.948010,27.956005
1350345,2006-01-30 20:12:46.100,45.960,27.995,20,ROMANIA,origin,M,4.1,INFO,None,45.960003,27.995008
